In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
# import pickle
import json
pd.options.mode.chained_assignment = None

In [4]:
with open('companies.json', 'r') as json_file:
    companies_arr = json.load(json_file)

with open('seen_brands.json', 'r') as json_file:
    seen_brands = json.load(json_file)

with open('unseen_brands.json', 'r') as json_file:
    unseen_brands = json.load(json_file)

In [5]:
train = pd.read_excel("content_simulation_train.xlsx")
test_time = pd.read_excel("content_simulation_test_time.xlsx")
test_company = pd.read_excel("content_simulation_test_company.xlsx")

## For unseen periods (test_time)

In [8]:
def unseen_brand(likes, input_date, inp_unseen_brand, k=5):
    input_date = datetime.strptime(input_date, '%Y-%m-%d %H:%M:%S')
    train['date'] = pd.to_datetime(train['date'])
    
    filtered_df = train.copy()
    
    companies = []
    for row in companies_arr:
        if inp_unseen_brand in row:
            companies = row
    
    filtered2_df = filtered_df[filtered_df['inferred company'].isin(companies)]
    filtered2_df['date_distance'] = abs(filtered2_df['date'] - input_date)
    filtered2_df['likes_distance'] = abs(filtered2_df['likes'] - likes)
    
    sorted_df = filtered2_df.sort_values(by=['date_distance', 'likes_distance'])
    top_k_df = sorted_df.head(k)
    top_k_df = top_k_df.drop(columns=['date_distance']).reset_index(drop=True)
    top_k_df = top_k_df.drop(columns=['likes_distance']).reset_index(drop=True)
    
    return top_k_df

In [7]:
def unseen_period(input_company_name,input_likes,k=5):
    matching_rows = train[train['inferred company'] == input_company_name]

    if matching_rows.empty:
        print(f"No matching rows found for {input_company_name}")
        return None

    df_new = matching_rows.copy()

    df_new['likes_diff'] = abs(train['likes'] - input_likes)

    df_sorted = df_new.sort_values(by='likes_diff')

    if(len(df_sorted)>=k):
        return df_sorted[:k]
    else:
        return df_sorted

In [106]:
type(unseen_brands),type(seen_brands)

(list, list)

In [6]:
train['inferred company'][10],test_time['inferred company'][5],test_company['inferred company'][100]

('mcafee', 'eaton', 'sky')

In [26]:
test_time['KNN 1'] = ""
test_time['KNN 2'] = ""
test_time['KNN 3'] = ""

In [27]:
for i in range(len(test_time)):
    brand = test_time['inferred company'][i]
    if(i%500==0):
        print(i)
    likes = test_time['likes'][i]
    df_output = unseen_period(brand,likes,3)

    try:
        triplet = df_output['content'].reset_index(drop=True).tolist()
        test_time['KNN 1'][i] = triplet[0]
        test_time['KNN 2'][i] = triplet[1]
        test_time['KNN 3'][i] = triplet[2]
    except:
        print(i)
        print(pd.Series(triplet,dtype='float64'))
    triplet = pd.Series([],dtype='float64')

0
500
1000
1500
No matching rows found for bulgari
1538
Series([], dtype: float64)
No matching rows found for bulgari
1978
Series([], dtype: float64)
2000
2500
No matching rows found for radioshack
2542
Series([], dtype: float64)
3000
3500
4000
4500
5000
No matching rows found for bulgari
5020
Series([], dtype: float64)
5500
6000
6500
No matching rows found for bulgari
6692
Series([], dtype: float64)
7000
No matching rows found for bulgari
7189
Series([], dtype: float64)
7500
No matching rows found for bulgari
7781
Series([], dtype: float64)
8000
8500
No matching rows found for radioshack
8616
Series([], dtype: float64)
9000
9500


In [28]:
test_company['KNN 1'] = ""
test_company['KNN 2'] = ""
test_company['KNN 3'] = ""

In [29]:
test_company['likes']

0          2
1          1
2        502
3        151
4        362
        ... 
9995     380
9996    2563
9997       1
9998     324
9999     167
Name: likes, Length: 10000, dtype: int64

In [30]:
not_working =[]
for i in range(len(test_company)):
    brand = test_company['inferred company'][i]
    if(i%500==0):    
        print(i)
    likes = test_company['likes'][i]
    date = test_company['date'][i]
    df_output = unseen_brand(likes, date, brand, 3)
    triplet = df_output['content'].reset_index(drop=True)
    try:
        test_company['KNN 1'][i] = triplet[0]
        test_company['KNN 2'][i] = triplet[1]
        test_company['KNN 3'][i] = triplet[2]
    except:
        print(i)
        not_working.append(i)
        print(pd.Series(triplet))

0
500
1000
1500
2000
2500


In [24]:
arr=[]
for i in not_working:
    arr.append(test_company['inferred company'][i])
arr2 = np.array(arr)
np.unique(arr2)

array([], dtype=float64)

In [25]:
test_company.to_csv("test_company.csv")

In [22]:
test_time.to_csv("test_time.csv")